# Pre-processing boston marathon Data-set

The aim of this is to process the three results available of boston marathon here:

    - https://www.kaggle.com/rojour/boston-results
    
With the objective of split our information in this couple of segments:
    - 5k
    - 10k
    - 15k
    - 20k
    - 25k
    - 30k
    - 35k
    - 40k
    

One each of this DataSet will include this factors:
    - tired level
    - temperature
    - humidity 
    - gender
    - age
    

In [17]:
import pandas as pd
import numpy as np

- First of all, add a label of which year is this results in the aim to diference the diferent runners.

In [18]:
results_2015 = pd.read_csv("data_sets/marathon_results_2015.csv")
results_2016 = pd.read_csv("data_sets/marathon_results_2016.csv")
results_2017 = pd.read_csv("data_sets/marathon_results_2017.csv")
segments = ['5K', '10K', '15K', 'Half', '25K', '30K', '35K', '40K']
results_2015['year'] = 2015
results_2016['year'] = 2016
results_2017['year'] = 2017

   - Then delete the rows which have '-' instead of a time. 

In [19]:
def clean_times(data):
    for segment in segments:
        data = data[data[segment] != '-']
    return data.copy()


results_2015 = clean_times(results_2015)
results_2016 = clean_times(results_2016)
results_2017 = clean_times(results_2017)


# Origin city elevation


- Our aim here is to obtain the elevation of the diferent cities, because the best runners are this who born in a city with a hight elevation.

- First of all we have to obtain the diferent cities on all years.

In [29]:
cities_2015 = results_2015['City'].unique()
cities_2016 = results_2016['City'].unique()
cities_2017 = results_2017['City'].unique()

In [49]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
location = geolocator.geocode(cities_2015[1])
print(location.raw['lon'])

38.7612525


In [36]:
import requests
import pandas as pd

# script for returning elevation from lat, long, based on open elevation data
# which in turn is based on SRTM
def get_elevation(lat, long):
    query = ('https://api.open-elevation.com/api/v1/lookup'
             f'?locations=%s,%s' % ((lat, long )))
    r = requests.get(query).json()  # json object, various ways you can extract value
    # one approach is to use pandas json functionality:
    elevation = pd.io.json.json_normalize(r, 'results')['elevation'].values[0]
    return elevation

get_elevation("42.361145", "-71.057083")

11

## Climate information using open weather map

https://darksky.net


+ GET request: ```https://api.darksky.net/forecast/{API-Key}/{latitude}, {longitude}, {Unix time}```


+ And our result will be a json with a lot of information, but we are only interested on this part : 

```"currently": {
    "time": 1429520400,
    "summary": "Partly Cloudy",
    "icon": "partly-cloudy-night",
    "precipIntensity": 0,
    "precipProbability": 0,
    "temperature": 41.45,
    "apparentTemperature": 36.18,
    "dewPoint": 38.25,
    "humidity": 0.88,
    "pressure": 1022.49,
    "windSpeed": 8.34,
    "windGust": 15.65,
    "windBearing": 120,
    "cloudCover": 0.36,
    "uvIndex": 0,
    "visibility": 10
  },```
  


In [20]:
import requests
import json

API_KEY = "YOUR API KEY HERE"
latitude = "42.361145"
longitude = "-71.057083"

UNIX_2015 = 1429518650
UNIX_2016 = 1429345850
UNIX_2017 = 1492417850

URI_2O15 = "https://api.darksky.net/forecast/%s/%s, %s, %s" % ((API_KEY, latitude ,longitude, UNIX_2015))
URI_2016 = "https://api.darksky.net/forecast/%s/%s, %s, %s" % ((API_KEY, latitude ,longitude, UNIX_2016))
URI_2017 = "https://api.darksky.net/forecast/%s/%s, %s, %s" % ((API_KEY, latitude ,longitude, UNIX_2017))

weather_2015 = requests.get(URI_2O15).json()['currently']
weather_2016 = requests.get(URI_2016).json()['currently']
weather_2017 = requests.get(URI_2017).json()['currently']

- add weather information to the datasets.

In [21]:
def add_weather_info(results_year, weather_year):
    results_year["humidity"] = weather_year["humidity"]
    results_year["temperature"] = weather_year["apparentTemperature"]
    
add_weather_info(results_2015, weather_2015)
add_weather_info(results_2016, weather_2016)
add_weather_info(results_2017, weather_2017)

## Split diferent segments

- Each datafrem has to be splitted in eight Data Frames 5K, 10K, 15K, 20K, 25K , 30K, 35K, 40K. Also transform time to seconds and at a new parameter, current_time. This will be useful when we will want the heat and humidity at certain moment.


In [ ]:
def get_sec(time_str):  
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def get_1km_partial(time, kms):
    pass

def segmentation(data_frame):
    df_segments = {}
    for segment in segments:
        df_segment = pd.DataFrame()
        df_segment['Name'] = data_frame['Name']
        df_segment['time_arrived'] = list(map(get_sec, data_frame[segment].values))
        df_segment['Current_time'] = 
        df_segment['temperature'] = df_segment['temperature']
        df_segment['Tired_level'] = 0
        df_segment['km_partial'] = 0
        df_segment['Gender'] = data_frame['M/F']
        df_segment['Age'] = data_frame['Age']
        df_segment['City'] = data_frame['City']
        df_segments[segment] = df_segment.copy()
    return df_segments

In [ ]:
segments_2015 = segmentation(results_2015)
segments_2016 = segmentation(results_2016)
segments_2017 = segmentation(results_2017)

 - Merge all the same segments of diferent years.

In [ ]:
def merge_segments(fragment_1, fragment_2, fragment_3):
    all_segments = {}
    for segment in segments:
        all_segments[segment] = pd.concat([fragment_1[segment], fragment_2[segment], fragment_3[segment]])
    return all_segments

In [ ]:
merge_segments(segments_2015, segments_2016, segments_2017)['5K']
